#Assignment #6

Vision Transformer

---


**SUBMISSION INSTRUCTIONS**

First make a copy of this colab file and then solve the assignment and upload your final notebook on github.

Before uploading your downloaded notebook, RENAME the file as rollno_name.ipynb

Submission Deadline : 31/01/2026 Saturday EOD i.e before 11:59 PM

The deadline is strict, Late submissions will incur penalty

Note that you have to upload your solution on the github page of the project Vision Transformer and **under Assignment 6**

And remember to keep title of your pull request to be ViT_name_rollno_assgn6

Github Submission repo -
https://github.com/electricalengineersiitk/Winter-projects-25-26/tree/main/Vision%20transformer/Assignment%206

# Part A - Data and tokens

Q1. Build a tiny toy dataset with pandas
Create a pandas DataFrame with columns text and label.
- Include at least 12 short sentences (3-10 words each).
- The label is 0/1 (e.g., positive vs negative sentiment).
- Shuffle rows and split into train/test (80/20) using a fixed random seed.
Return: df_train, df_test.

In [9]:
import pandas as pd
def make_toy_dataset(seed: int = 42):
    """Return df_train, df_test with columns: text (str), label (int)."""
    data = [
        ("the movie was great", 1), ("i loved the acting", 1),
        ("amazing plot and characters", 1), ("highly recommended for everyone", 1),
        ("a masterpiece of cinema", 1), ("best film this year", 1),
        ("the plot was boring", 0), ("i hated the script", 0),
        ("worst experience ever", 0), ("do not watch this", 0),
        ("very disappointing ending", 0), ("waste of time and money", 0)
    ]

    df = pd.DataFrame(data, columns=['text', 'label'])

    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    split_idx = int(len(df) * 0.8)
    df_train = df.iloc[:split_idx].copy()
    df_test = df.iloc[split_idx:].copy()
    return df_train, df_test

Q2. Clean and tokenize text

Implement a basic cleaner: lowercase, strip, replace multiple spaces with one, and remove punctuation
(.,!?;:).
Tokenize by whitespace.
Add a new column tokens that stores a list of tokens per row.
Return the updated DataFrame.

In [10]:
import re
import pandas as pd
def clean_text(s: str) -> str:
 s = s.lower().strip()
 s = re.sub(r'[.,!?:;]', '', s)  # Remove specific punctuation
 s = re.sub(r'\s+', ' ', s)      # Replace multiple spaces with one
 return s
def add_tokens_column(df: pd.DataFrame) -> pd.DataFrame:
 """Adds df['tokens'] = list[str]."""
 df['tokens'] = df['text'].apply(lambda x: clean_text(x).split())
 return df


Q3. Build a vocabulary + token/id mappings

Build token2id and id2token using the training tokens.
Include special tokens: [PAD], [UNK], [BOS], [EOS] at the beginning.
Add tokens that occur at least min_freq times.
Return: token2id (dict), id2token (list).

In [11]:
from collections import Counter
from typing import Dict, List
SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']
def build_vocab(list_of_token_lists, min_freq: int = 1):
 SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']

 counts = Counter([tok for sublist in list_of_token_lists for tok in sublist])

 vocab = [tok for tok, freq in counts.items() if freq >= min_freq]
 id2token = SPECIALS + sorted(vocab)
 token2id = {tok: i for i, tok in enumerate(id2token)}
 return token2id, id2token

Q4. Convert tokens to ids + pad to a batch

Implement tokens_to_ids for one sequence.
Implement pad_batch that takes a list of id sequences and returns:
- X: int array (B,T) padded with pad_id
- pad_mask: bool array (B,T) where True means 'real token' and False means padding

In [17]:
import numpy as np
def tokens_to_ids(tokens, token2id, unk_token='[UNK]'):
 ids = [token2id.get(tok, token2id[unk_token]) for tok in tokens]
 return ids
def pad_batch(id_seqs, pad_id: int):
 """Return X (B,T) and pad_mask (B,T)"""
 max_len = max(len(seq) for seq in id_seqs)
 batch_size = len(id_seqs)
 X = np.full((batch_size, max_len), pad_id)
 pad_mask = np.zeros((batch_size, max_len), dtype=bool)

 for i, seq in enumerate(id_seqs):
     X[i, :len(seq)] = seq
     pad_mask[i, :len(seq)] = True
     return X, pad_mask


#Part B - Core Transformer math

Q5. Embedding lookup

Implement an embedding table E of shape (V,D) initialized from a normal distribution (mean 0, std 0.02).
Given token ids X (B,T), return embeddings of shape (B,T,D) using NumPy indexing.


In [18]:
import numpy as np
def init_embeddings(vocab_size: int, d_model: int, seed: int = 0):
 np.random.seed(seed)
 return np.random.normal(0, 0.02, (vocab_size, d_model))
def embed(X: np.ndarray, E: np.ndarray):
 """X: (B,T) int, E: (V,D) -> out: (B,T,D)."""
 return E[X]


Q6. Sinusoidal positional encoding

Implement the classic sinusoidal positional encoding PE of shape (T,D).
Then add it to token embeddings (B,T,D).
Make sure your implementation works for both even and odd D.

In [19]:
import numpy as np
def sinusoidal_positional_encoding(T: int, D: int):
 """Return PE: (T,D)."""
 PE = np.zeros((T, D))
 position = np.arange(T)[:, np.newaxis]
 div_term = np.exp(np.arange(0, D, 2) * -(np.log(10000.0) / D))
 PE[:, 0::2] = np.sin(position * div_term)
 if D % 2 == 1:
    PE[:, 1::2] = np.cos(position * div_term[:D//2])
 else:
    PE[:, 1::2] = np.cos(position * div_term)
 return PE
def add_positional_encoding(X_emb: np.ndarray, PE: np.ndarray):
 """X_emb: (B,T,D), PE: (T,D) -> (B,T,D)."""
 return X_emb + PE[:X_emb.shape[1], :]


Q7. Scaled dot-product attention with masking

Implement scaled dot-product attention:
Attention(Q,K,V) = softmax((Q @ K^T) / sqrt(dk) + mask) @ V
Inputs: Q,K,V are (B,H,T,Dh). Mask is boolean broadcastable to (B,H,T,T) where False means 'mask out'.
Requirements:
- Use a numerically stable softmax (subtract max).
- Convert boolean mask to large negative values before softmax.
Return: context (B,H,T,Dh) and attention weights (B,H,T,T).

In [22]:
import numpy as np
def softmax(x: np.ndarray, axis: int = -1):
 x_max = np.max(x, axis=axis, keepdims=True)
 exp_x = np.exp(x - x_max)
 return exp_x / np.sum(exp_x, axis=axis, keepdims=True)
def scaled_dot_product_attention(Q, K, V, mask=None):
 """Q,K,V: (B,H,T,Dh), mask: bool broadcastable to (B,H,T,T)."""
 dk = Q.shape[-1]
 scores = (Q @ K.transpose(0, 1, 3, 2)) / np.sqrt(dk)

 if mask is not None:
     scores = np.where(mask == False, -1e9, scores)
     attn = softmax(scores, axis=-1)
     context = attn @ V

 return context, attn

Q8. Multi-head self-attention (MHA)

Implement multi-head self-attention for input X (B,T,D).
- Project to Q,K,V using weight matrices Wq,Wk,Wv each (D,D).
- Reshape/split into heads -> (B,H,T,Dh) where Dh=D/H.
- Apply scaled dot-product attention with a pad mask (B,T) (broadcast it appropriately).
- Concatenate heads and apply output projection Wo (D,D).
Return: out (B,T,D) and attention weights (B,H,T,T).

In [24]:
import numpy as np
def linear(x: np.ndarray, W: np.ndarray, b=None):
 y = x.dot(W)
 if b is not None:
      y += b
 return y
def split_heads(x: np.ndarray, n_heads: int):
 """(B,T,D) -> (B,H,T,Dh)"""
 B, T, D = x.shape
 Dh = D // n_heads
 return x.reshape(B, T, n_heads, Dh).transpose(0, 2, 1, 3)
def combine_heads(xh: np.ndarray):
 """(B,H,T,Dh) -> (B,T,D)"""
 B, H, T, Dh = xh.shape
 return xh.transpose(0, 2, 1, 3).reshape(B, T, H * Dh)
def mha_self_attention(X, Wq, Wk, Wv, Wo, n_heads: int, pad_mask=None):
 Q = split_heads(linear(X, Wq), n_heads)
 K = split_heads(linear(X, Wk), n_heads)
 V = split_heads(linear(X, Wv), n_heads)
 mask = pad_mask[:, None, None, :] if pad_mask is not None else None
 context_h, attn = scaled_dot_product_attention(Q, K, V, mask)
 out = linear(combine_heads(context_h), Wo)
 return out, attn

Q9. LayerNorm + residual connection

Implement LayerNorm for X (B,T,D) using learnable gamma and beta of shape (D,).
Then implement residual_add_and_norm(Y, X, gamma, beta) that returns LayerNorm(X + Y).

In [25]:
import numpy as np
def layer_norm(X: np.ndarray, gamma: np.ndarray, beta: np.ndarray, eps: float = 1e-5):
 mean = x.mean(axis=-1, keepdims=True)
 std = x.std(axis=-1, keepdims=True)
 Y = gamma * (x - mean) / (std + eps) + beta
 return Y
def residual_add_and_norm(Y: np.ndarray, X: np.ndarray, gamma: np.ndarray, beta: np.ndarray):
 Z = layer_norm(X + Y, gamma, beta)
 return Z

Q10. Position-wise FeedForward network

Implement FFN: FFN(X) = relu(X @ W1 + b1) @ W2 + b2
Shapes: X (B,T,D), W1 (D,Dff), b1 (Dff,), W2 (Dff,D), b2 (D,)
Return: (B,T,D).

In [26]:
import numpy as np
def relu(x: np.ndarray):
 y = np.maximum(0, x)
 return y
def feed_forward(X: np.ndarray, W1: np.ndarray, b1: np.ndarray, W2: np.ndarray, b2: np.ndarray):
 h = relu(linear(X, W1, b1))
 Y = linear(h, W2, b2)
 return Y

# Part C - Putting it together

Q11. One Transformer encoder block (forward)

Implement a single encoder block forward pass:
1) MHA = MultiHeadSelfAttention(X) with pad_mask
2) X1 = LayerNorm(X + MHA)
3) FFN = FeedForward(X1)
4) X2 = LayerNorm(X1 + FFN)
Return X2.
You may pass all parameters explicitly (weights, gamma/beta).

In [27]:
def encoder_block_forward(X, params, n_heads: int, pad_mask=None):
 """X: (B,T,D). params is a dict of numpy arrays."""
 mha_out, _ = mha_self_attention(X, params['Wq'], params['Wk'], params['Wv'],params['Wo'], n_heads, pad_mask)
 X1 = residual_add_and_norm(mha_out, X, params['gamma1'], params['beta1'])
 ffn_out = feed_forward(X1, params['W1'], params['b1'], params['W2'], params['b2'])
 X2 = residual_add_and_norm(ffn_out, X1, params['gamma2'], params['beta2'])
 return X2

Q12. Sequence classification head + end-to-end demo

Create an end-to-end forward pass for a tiny classifier:
- Input ids -> embeddings + positional enc
- One encoder block
- Pooling: take the [BOS] position (t=0) as the sequence representation
- Linear head: logits = h0 @ Wcls + bcls with Wcls (D,2), bcls (2,)
- Softmax to probabilities
Write predict_proba that takes a batch of texts and returns probs (B,2).
Include simple sanity checks: shapes, probabilities sum to 1, and masking doesn't crash for different
lengths.


In [29]:
def predict_proba(texts, token2id, E, PE, params, Wcls, bcls, n_heads: int):
 """Return probs: (B,2)."""
 cleaned = [clean_text(t).split() for t in texts]
 seqs = [['[BOS]'] + s + ['[EOS]'] for s in cleaned]
 ids_list = [tokens_to_ids(s, token2id) for s in seqs]
 X_ids, mask = pad_batch(ids_list, token2id['[PAD]'])
 X_emb = embed(X_ids, E)
 X_pos = add_positional_encoding(X_emb, PE)
 X_enc = encoder_block_forward(X_pos, params, n_heads, mask )
 return probs